# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

## Construindo Um Algoritmo Para Rede Neural Multilayer Perceptron

### Função de Ativação Sigmóide

As transformações lineares são ótimas para simplesmente mudar os valores, mas as redes neurais muitas vezes exigem uma transformação mais específica. Por exemplo, um dos desenhos originais para um neurônio artificial, o perceptron, exibe comportamento de saída binária. Perceptrons compararam uma entrada ponderada com um limite. Quando a entrada ponderada excede o limite, o perceptron é ativado e a saída é igual a 1, caso contrário, ele produz 0.

A ativação (nome dado para o comportamento de saída binária) geralmente faz sentido para problemas de classificação. Por exemplo, se você pedir à rede que elabore uma hipótese se uma imagem manuscrita é um '7', você está realmente perguntando por uma saída binária - sim, isso é um '7', ou não, isso não é um '7'. Uma função de etapa (Step Function) é a forma mais rápida de uma saída binária, o que é excelente. Mas as Step Functions não são contínuas e não diferenciáveis, o que é muito ruim. A diferenciação é o que torna possível a descida do gradiente.

A função sigmóide, substitui o limiar com uma bela curva em forma de S que imita o comportamento de ativação de um perceptron enquanto é diferenciável. Como um bônus: a função sigmóide possui uma derivada muito simples que pode ser calculada a partir da própria função sigmóide.

Observe que a função sigmóide possui apenas um parâmetro. Lembre-se de que sigmóide é uma função de ativação (não-linearidade), o que significa que ela recebe uma única entrada e executa uma operação matemática nesta entrada.

Conceitualmente, a função sigmóide toma decisões. De acordo com os atributos ponderados (entradas x pesos), a função sigmóide indica se os atributos contribuem ou não para uma classificação. Desta forma, uma ativação sigmóide funciona bem seguindo uma transformação linear. Como usamos pesos e bias aleatórios, a saída do nó sigmóide também é aleatória. O processo de aprendizagem através de backpropagation e gradient descent, que você implementará em breve, modifica os pesos e o bias, de modo que a ativação do nó sigmóide comece a combinar as saídas esperadas.

In [1]:
import numpy as np

class Neuronio(object):
    def __init__(self, nodes_entrada = []):
        self.nodes_entrada = nodes_entrada
        self.valor = None
        self.nodes_saida = []
        for n in nodes_entrada:
            n.nodes_saida.append(self)

    def forward():
        raise NotImplementedError


class Input(Neuronio):
    def __init__(self):
        Neuronio.__init__(self)

    def forward(self):
        pass


class Linear(Neuronio):
    def __init__(self, X, W, b):
        Neuronio.__init__(self, [X, W, b])

    def forward(self):
        X = self.nodes_entrada[0].valor
        W = self.nodes_entrada[1].valor
        b = self.nodes_entrada[2].valor
        self.valor = np.dot(X, W) + b


class Sigmoid(Neuronio):
    def __init__(self, node):
        Neuronio.__init__(self, [node])

    def _sigmoid(self, x):
        return 1. / (1. + np.exp(-x)) 

    def forward(self):
        input_value = self.nodes_entrada[0].valor
        self.valor = self._sigmoid(input_value)


def topological_sort(feed_dict):
    """
    Classifica os nós em ordem topológica usando o Algoritmo de Kahn.

    `Feed_dict`: um dicionário em que a chave é um nó ` Input` e o valor é o respectivo feed de valor para esse nó.

    Retorna uma lista de nós ordenados.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.nodes_saida:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.valor = feed_dict[n]

        L.append(n)
        for m in n.nodes_saida:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_pass(output_node, sorted_nodes):
    """
   Executa uma passagem para a frente através de uma lista de nós ordenados.

     Argumentos:

         `Output_node`: Um nó no grafo, deve ser o nó de saída.
         `Sorted_nodes`: uma lista topologicamente ordenada de nós.

     Retorna o valor do Nó de saída
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.valor

Pode parecer estranho que _sigmoid seja um método separado dentro da classe Sigmoid. A função sigmoid é realmente uma parte de sua própria derivada. Manter _sigmoid separado, significa que você não terá que implementá-lo duas vezes durante a fase de backpropagation.

### Executando o Grafo

In [2]:
# Este script cria e executa o grafo 

import numpy as np

# Parâmetros de entrada
inputs, weights, bias = Input(), Input(), Input()

# Função Linear
f = Linear(inputs, weights, bias)
g = Sigmoid(f)

# Atribuindo valores aos parâmetros
x = np.array([[-2., -1.], [-2, -4]])
w = np.array([[4., -6], [3., -2]])
b = np.array([-2., -3])

# Define o feed_dict
feed_dict = {inputs: x, weights: w, bias: b}

# Ordena as entradas para execução
graph = topological_sort(feed_dict)

# Gera o output com o forward_pass. Perceba que usamos a função g e não a f aqui!!!!!
output = forward_pass(g, graph)

# Print
print(output)

[[2.26032430e-06 9.99983299e-01]
 [2.78946809e-10 9.99999959e-01]]


## Fim